# 3. Creating a multi-UDF Agent: Vancouver Open Data Agent

In this notebook we'll build a multi-UDF Agent that can have access to a few UDFs that fetch information from [Vancouver's Open Data portal](https://opendata.vancouver.ca/pages/home/)

In [1]:
import fused
import json
import os
import time
from pathlib import Path

In [2]:
# We still need your local paths
PATH_TO_CLAUDE_CONFIG = (
    f"{str(Path.home())}/Library/Application Support/Claude/claude_desktop_config.json"
)


if not os.path.exists(PATH_TO_CLAUDE_CONFIG):
    # Creating the config file
    os.makedirs(os.path.dirname(PATH_TO_CLAUDE_CONFIG), exist_ok=True)
    with open(PATH_TO_CLAUDE_CONFIG, "w") as f:
        json.dump({}, f)

assert os.path.exists(PATH_TO_CLAUDE_CONFIG), (
    "Please update the PATH_TO_CLAUDE_CONFIG variable with the correct path to your Claude config file"
)

In [3]:
# Local path to the Claude app
CLAUDE_APP_PATH = "/Applications/Claude.app"
assert os.path.exists(CLAUDE_APP_PATH), (
    "Please update the CLAUDE_APP_PATH variable with the correct path to your Claude app"
)

In [4]:
# Change this path if you're not running this from the repo root
WORKING_DIR = os.getcwd()

In [48]:
# We'll load the commons folder once again to have our helper functions
commit = "5dda36c"
common = fused.load(
    f"https://github.com/fusedio/udfs/tree/{commit}/public/common"
).utils

In [6]:
# And see which agents we have available
json.load(open(os.path.join(WORKING_DIR, "agents.json")))

{'agents': [{'name': 'get_current_time', 'udfs': ['current_utc_time']},
  {'name': 'fused_docs', 'udfs': ['list_public_udfs', 'reading_fused_docs']}]}

We'll make 4 UDFs:
- Returning the location of 100 EV chargers in the city
- Returning location, name & size of 100 parks in Vancouver
- Returning location & details of 100 latest building permits in Vancouver
- Returning the internet speed of any lat / lon (not just in Vancouver, but this also works for Vancouver area)

In [44]:
AGENT_NAME = "vancouver_open_data"

In [19]:
@fused.udf
def parks_vancouver():
    """
    UDF to get the polygon geometries of parks in Vancouver based on Open Data Portal
    """
    import requests
    import geopandas as gpd
    import pandas as pd
    from shapely.geometry import Polygon, MultiPolygon, shape
    import numpy as np
    from pandas import json_normalize

    @fused.cache
    def get_request(url):
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response

    limit = 100
    parks_url = f"https://opendata.vancouver.ca/api/explore/v2.1/catalog/datasets/parks-polygon-representation/records?limit={str(limit)}"
    response = get_request(url=parks_url)
    json_data = response.json()
    
    # First extract all non-geometry data
    df = json_normalize(json_data['results'])
    
    # Drop the geom column which will be replaced with proper geometry
    if 'geom' in df.columns:
        df = df.drop(columns=['geom'])

    # Filter for valid polygons or multipolygons
    valid_items = [
        item for item in json_data['results']
        if 'geom' in item 
        and item['geom'] is not None
        and isinstance(item['geom'], dict)
        and 'geometry' in item['geom'] 
        and item['geom']['geometry'] is not None
        and isinstance(item['geom']['geometry'], dict)
        and 'type' in item['geom']['geometry']
        and item['geom']['geometry']['type'] in ['Polygon', 'MultiPolygon']
    ]
    
    # Create geometries using shapely's shape function
    geometries = []
    for item in valid_items:
        try:
            geom = shape(item['geom']['geometry'])
            geometries.append(geom)
        except Exception as e:
            print(f"Error creating geometry: {e}")
            continue
    
    # Create filtered dataframe with matching indices
    filtered_df = pd.DataFrame(valid_items).drop(columns=['geom'])
    
    # Create GeoDataFrame with geometries
    gdf = gpd.GeoDataFrame(
        filtered_df, 
        geometry=geometries,
        crs="EPSG:4326"
    )
    # gdf['area'] = gdf.geometry.area
    print(f"{gdf.sample()=}")
    print(f"{gdf.columns=}")
    print(gdf.shape)
    return gdf

In [20]:
fused.run(parks_vancouver)

,park_id,park_name,area_ha,park_url,geo_point_2d,geometry
0,108.0,Connaught Park,5.993645,http://covapp.vancouver.ca/parkfinder/parkdeta...,"{'lat': 49.26205550109892, 'lon': -123.1601050...","POLYGON ((-123.1623 49.26292, -123.15784 49.26..."
1,81.0,Clark Park,4.295203,http://covapp.vancouver.ca/parkfinder/parkdeta...,"{'lat': 49.25710409057022, 'lon': -123.0723570...","POLYGON ((-123.07002 49.25766, -123.07004 49.2..."
2,37.0,Chaldecott Park,3.454305,http://covapp.vancouver.ca/parkfinder/parkdeta...,"{'lat': 49.249047303072075, 'lon': -123.192237...","POLYGON ((-123.19347 49.24991, -123.19099 49.2..."
3,174.0,Braemar Park,1.258929,http://covapp.vancouver.ca/parkfinder/parkdeta...,"{'lat': 49.24762496945391, 'lon': -123.1235807...","POLYGON ((-123.12462 49.24801, -123.12251 49.2..."
4,236.0,Ebisu Park,0.420569,http://covapp.vancouver.ca/parkfinder/parkdeta...,"{'lat': 49.20538526928567, 'lon': -123.1324784...","POLYGON ((-123.13189 49.20509, -123.13247 49.2..."
...,...,...,...,...,...,...
95,8.0,Trafalgar Park,4.860801,http://covapp.vancouver.ca/parkfinder/parkdeta...,"{'lat': 49.25097540413386, 'lon': -123.1624811...","POLYGON ((-123.16514 49.25106, -123.16512 49.2..."
96,188.0,Ross Park,1.511835,http://covapp.vancouver.ca/parkfinder/parkdeta...,"{'lat': 49.21726674556697, 'lon': -123.0823563...","POLYGON ((-123.08296 49.21803, -123.08173 49.2..."
97,140.0,Robson Park,1.563992,http://covapp.vancouver.ca/parkfinder/parkdeta...,"{'lat': 49.258160306611416, 'lon': -123.092024...","POLYGON ((-123.09127 49.25792, -123.09113 49.2..."
98,168.0,Riley Park,2.703745,http://covapp.vancouver.ca/parkfinder/parkdeta...,"{'lat': 49.24229347353439, 'lon': -123.1042653...","POLYGON ((-123.1051 49.24326, -123.10334 49.24..."


In [23]:
parks_mcp_metadata = {
    "description": "This UDF returns the location of some (not all) the parks in the Vancouver area as a GeoDataFrame. This contains: The polygon of those parks, The area of the park, The name of the park",
    "parameters": [
        {
            "name": "",
            "type": "",
        }
    ],
}

parks_mcp_metadata

{'description': 'This UDF returns the location of some (not all) the parks in the Vancouver area as a GeoDataFrame. This contains: The polygon of those parks, The area of the park, The name of the park',
 'parameters': [{'name': '', 'type': ''}]}

In [24]:
# Adding our UDF + MCP_metadata to our Agent
common.save_to_agent(
    agent_json_path=os.path.join(WORKING_DIR, "agents.json"),
    udf=parks_vancouver,
    agent_name=AGENT_NAME,
    udf_name="hundred_parks_in_vancouver",
    mcp_metadata=parks_mcp_metadata,
)

In [25]:
@fused.udf
def ev_chargers():
    """
    UDF to get the location of electric chargers around Vancouver based on Open Data Portal
    """
    import requests
    import geopandas as gpd
    import pandas as pd
    from shapely.geometry import Point
    import numpy as np
    from pandas import json_normalize

    @fused.cache
    def get_request(url):
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response

    limit = 100
    building_permits_url = f"https://opendata.vancouver.ca/api/explore/v2.1/catalog/datasets/electric-vehicle-charging-stations/records?limit={str(limit)}"
    response = get_request(url=building_permits_url)
    json_data = response.json()
    
    # First extract all non-geometry data
    df = json_normalize(json_data['results'])
    
    # Drop the geom column which will be replaced with proper geometry
    if 'geom' in df.columns:
        df = df.drop(columns=['geom'])

    # Skipping any point that doesn't have valid geom
    valid_items = [
        item for item in json_data['results']
        if 'geom' in item 
        and item['geom'] is not None
        and isinstance(item['geom'], dict)
        and 'geometry' in item['geom'] 
        and item['geom']['geometry'] is not None
        and isinstance(item['geom']['geometry'], dict)
        and 'type' in item['geom']['geometry']
        and item['geom']['geometry']['type'] == 'Point'
    ]
    
    # Extract coordinates directly into arrays
    coords = np.array([
        item['geom']['geometry']['coordinates'] 
        for item in valid_items
    ])
    
    # Create Points in a vectorized way
    geometries = [Point(x, y) for x, y in coords]
    
    # Create filtered dataframe with matching indices
    filtered_df = pd.DataFrame(valid_items).drop(columns=['geom'])
    
    # Create GeoDataFrame with geometries
    gdf = gpd.GeoDataFrame(
        filtered_df, 
        geometry=geometries,
        crs="EPSG:4326"
    )
    print(gdf.shape)
    return gdf
    


In [26]:
# We can run this UDF locally with `fused.run(udf)`
fused.run(ev_chargers)

,address,lot_operator,geo_local_area,geo_point_2d,geometry
0,Beach Ave. @ Cardero St,Easy Park / Park Board,West End,"{'lat': 49.283155, 'lon': -123.142173}",POINT (-123.14217 49.28316)
1,1 Kingsway,Easypark,Mount Pleasant,"{'lat': 49.2641594, 'lon': -123.1002054}",POINT (-123.10021 49.26416)
2,4575 Clancy Loranger Way,Park Board,Riley Park,"{'lat': 49.243665, 'lon': -123.106578}",POINT (-123.10658 49.24366)
3,845 Avison Way,Vancouver Aquarium,None,"{'lat': 49.299534, 'lon': -123.13022}",POINT (-123.13022 49.29953)
4,273 E 53rd Ave,City of Vancouver,Sunset,"{'lat': 49.222235022762, 'lon': -123.100095218...",POINT (-123.1001 49.22224)
5,3311 E. Hastings,City of Vancouver,Hastings-Sunrise,"{'lat': 49.281369, 'lon': -123.033786}",POINT (-123.03379 49.28137)
6,959-979 Mainland St,City of Vancouver,Downtown,"{'lat': 49.2769232201811, 'lon': -123.11926106...",POINT (-123.11926 49.27692)
7,451 Beach Crescent,City of Vancouver,Downtown,"{'lat': 49.272514, 'lon': -123.12833}",POINT (-123.12833 49.27251)
8,646 E. 44th Ave,City of Vancouver,Sunset,"{'lat': 49.229928611422, 'lon': -123.09135336453}",POINT (-123.09135 49.22993)
9,5175 Dumfries St,City of Vancouver,Kensington-Cedar Cottage,"{'lat': 49.2385063171386, 'lon': -123.07548522...",POINT (-123.07549 49.23851)


In [29]:
ev_charger_mcp_metadata = {
    "description": "This UDF returns the location of all the electric chargers in Vancouver as a GeoDataFrame with the name of the chargers and their lat lon",
    "parameters": [
        {
            "name": "",
            "type": "",
        }
    ],
}

ev_charger_mcp_metadata

{'description': 'This UDF returns the location of all the electric chargers in Vancouver as a GeoDataFrame with the name of the chargers and their lat lon',
 'parameters': [{'name': '', 'type': ''}]}

In [32]:
# We add this new UDF + mcp_metadata to the same agent
common.save_to_agent(
    agent_json_path=os.path.join(WORKING_DIR, "agents.json"),
    udf=ev_chargers,
    agent_name=AGENT_NAME,
    udf_name="electric_vehicle_chargers_in_vancouver",
    mcp_metadata=ev_charger_mcp_metadata,
)

In [34]:
@fused.udf
def built_permits():
    """
    Returns 100 latest built permits in Vancouver area
    """
    import requests
    import geopandas as gpd
    import pandas as pd
    from shapely.geometry import Point
    import numpy as np
    from pandas import json_normalize

    @fused.cache
    def get_request(url):
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response

    limit = 100
    building_permits_url = f"https://opendata.vancouver.ca/api/explore/v2.1/catalog/datasets/issued-building-permits/records?limit={str(limit)}"
    response = get_request(url=building_permits_url)
    json_data = response.json()
    
    # First extract all non-geometry data
    df = json_normalize(json_data['results'])
    
    # Drop the geom column which will be replaced with proper geometry
    if 'geom' in df.columns:
        df = df.drop(columns=['geom'])

    # Skipping any point that doesn't have valid geom
    valid_items = [
        item for item in json_data['results']
        if 'geom' in item 
        and item['geom'] is not None
        and isinstance(item['geom'], dict)
        and 'geometry' in item['geom'] 
        and item['geom']['geometry'] is not None
        and isinstance(item['geom']['geometry'], dict)
        and 'type' in item['geom']['geometry']
        and item['geom']['geometry']['type'] == 'Point'
    ]
    
    # Extract coordinates directly into arrays
    coords = np.array([
        item['geom']['geometry']['coordinates'] 
        for item in valid_items
    ])
    
    # Create Points in a vectorized way
    geometries = [Point(x, y) for x, y in coords]
    
    # Create filtered dataframe with matching indices
    filtered_df = pd.DataFrame(valid_items).drop(columns=['geom'])
    
    # Create GeoDataFrame with geometries
    gdf = gpd.GeoDataFrame(
        filtered_df, 
        geometry=geometries,
        crs="EPSG:4326"
    )
    print(gdf.shape)
    return gdf

In [35]:
fused.run(built_permits)

,permitnumber,permitnumbercreateddate,issuedate,permitelapseddays,projectvalue,typeofwork,address,projectdescription,permitcategory,applicant,applicantaddress,propertyuse,specificusecategory,buildingcontractor,buildingcontractoraddress,issueyear,geolocalarea,yearmonth,geo_point_2d,geometry
0,BP-2021-03710,2021-07-16,2022-04-27,285,15000.0,Demolition / Deconstruction,"2626 W 35TH AVENUE, Vancouver, BC V6N 2L8",Low Density Housing - Demolition / Deconstruct...,None,Danny Lung & Sharon Chen DBA: Lung Designs Gro...,LUNG DESIGNS GROUP LTD\r\nUNIT 268-2633 VIKING...,[Dwelling Uses],[Single Detached House],Canadian Excavating Ltd,"6898 130 St\r\nSurrey, BC V3W 4J5",2022,Arbutus Ridge,2022-04,"{'lat': 49.2401088, 'lon': -123.1643722}",POINT (-123.16437 49.24011)
1,DB-2021-03858,2021-07-21,2022-02-22,216,1120192.5,New Building,"1638 W 62ND AVENUE, Vancouver, BC V6P 2G2",Low Density Housing - New Building - To constr...,New Build - Low Density Housing,Danny Lung & Sharon Chen DBA: Lung Designs Gro...,LUNG DESIGNS GROUP LTD\r\nUNIT 268-2633 VIKING...,[Dwelling Uses],[Single Detached House w/Sec Suite],D & P Development Corp.,"2588 W 21ST AV \r\nVancouver, BC V6L 1K1",2022,Marpole,2022-02,"{'lat': 49.2147246, 'lon': -123.1445553}",POINT (-123.14456 49.21472)
2,DB-2021-03708,2021-07-16,2022-05-05,293,1150532.5,New Building,"2626 W 35TH AVENUE, Vancouver, BC V6N 2L8",Low Density Housing - New Building - To constr...,New Build - Low Density Housing,Danny Lung & Sharon Chen DBA: Lung Designs Gro...,LUNG DESIGNS GROUP LTD\r\nUNIT 268-2633 VIKING...,[Dwelling Uses],[Single Detached House],Avis Homes Limited,"10388 Blundell Road \r\nRichmond, BC V6Y 1L1",2022,Arbutus Ridge,2022-05,"{'lat': 49.2401088, 'lon': -123.1643722}",POINT (-123.16437 49.24011)
3,BU467421,2016-01-07,2017-05-04,483,15000.0,Demolition / Deconstruction,"4261 W 13TH AVENUE, Vancouver, BC V6R 2T7",Low Density Housing - Demolition / Deconstruct...,None,Danny Lung & Sharon Chen DBA: Lung Designs Gro...,LUNG DESIGNS GROUP LTD\r\nUNIT 268-2633 VIKING...,[Dwelling Uses],[Single Detached House],Canadian Excavating Ltd,"6898 130 St\r\nSurrey, BC V3W 4J5",2017,West Point Grey,2017-05,"{'lat': 49.2612479, 'lon': -123.2019411}",POINT (-123.20194 49.26125)
4,DB-2017-05778,2017-11-08,2018-06-28,232,143872.5,New Building,"1788 E 47TH AVENUE, Vancouver, BC V5P 1P8",Low Density Housing - New Building - To constr...,New Build - Standalone Laneway,Danny Lung & Sharon Chen DBA: Lung Designs Gro...,LUNG DESIGNS GROUP LTD\r\nUNIT 268-2633 VIKING...,[Dwelling Uses],[Laneway House],Trustful Construction Ltd,"2779 W 13TH AV \r\nVancouver, BC V6K 2T5",2018,Victoria-Fraserview,2018-06,"{'lat': 49.2276219, 'lon': -123.0688679}",POINT (-123.06887 49.22762)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,BP-2022-01404,2022-03-17,2023-06-08,448,0.0,Salvage and Abatement,"584 W 24TH AVENUE, Vancouver, BC V5Z 2B4",Enquiry Centre - Salvage and Abatement - S&A\r...,None,Michael Meszaros DBA: Pioneer Code Consultants...,"1933 W Broadway\r\nVancouver, BC V6J 1Z3",[Dwelling Uses],[Single Detached House],Citrine Homes Ltd,"2916 W BROADWAY \r\nVancouver, BC V6K 2G8",2023,South Cambie,2023-06,"{'lat': 49.2497681, 'lon': -123.1175961}",POINT (-123.1176 49.24977)
95,BP-2020-01185,2020-03-31,2021-05-19,414,12136000.0,New Building,"8311 FRENCH STREET, Vancouver, BC V6P 4W3",Certified Professional Program - New Building ...,None,Michael Meszaros DBA: Pioneer Code Consultants...,"1933 W Broadway\r\nVancouver, BC V6J 1Z3",[Dwelling Uses],[Multiple Dwelling],Vanwell Homes Ltd,"1777 E 47TH AV \r\nVancouver, BC V5P 1P9",2021,Marpole,2021-05,"{'lat': 49.210493, 'lon': -123.1392022}",POINT (-123.1392 49.21049)
96,BP-2022-01445,2022-03-21,2023-01-10,295,0.0,New Building,"8337 FRENCH STREET, Vancouver, BC V6P 4W3",High Density Housing / Commercial - New Buildi...,None,Michael Meszaros DBA: Pioneer Code Consultants...,"1933 W Broadway\r\nVancouver, BC V6J 1Z3",[Dwelling Uses],[Dwelling Unit]

In [36]:
built_permits_mcp_metadata = {
    "description": "This UDF returns 100 built permits for the city of Vancouver. This includes the location of the permits, for how much money they were granted and when",
    "parameters": [
        {
            "name": "",
            "type": "",
        }
    ],
}

built_permits_mcp_metadata

{'description': 'This UDF returns 100 built permits for the city of Vancouver. This includes the location of the permits, for how much money they were granted and when',
 'parameters': [{'name': '', 'type': ''}]}

In [37]:
# We add this new UDF + mcp_metadata to the same agent
common.save_to_agent(
    agent_json_path=os.path.join(WORKING_DIR, "agents.json"),
    udf=built_permits,
    agent_name=AGENT_NAME,
    udf_name="building_permits_in_vancouver",
    mcp_metadata=built_permits_mcp_metadata,
)

In [38]:
@fused.udf
def ookla_internet_speed(bounds: fused.types.Bounds=None, lat: float=37.7749, lon: float=-122.4194):
    
    file_path='s3://ookla-open-data/parquet/performance/type=mobile/year=2024/quarter=3/2024-07-01_performance_mobile_tiles.parquet'
    
    # Load pinned versions of utility functions.
    utils = fused.load("https://github.com/fusedio/udfs/tree/ee9bec5/public/common/").utils
    
    # Sample usage: Set default lat/lon for San Francisco if none provided
    if lat is None and lon is None and bounds is None:
        print("Using sample coordinates for San Francisco")
        lat = 37.7749
        lon = -122.4194
    
    # Check if we're using point query or bounds
    if lat is not None and lon is not None:
        # Create a small bounding box around the input lat/lon
        buffer = 0.01  # ~1km at equator
        total_bounds = [lon - buffer, lat - buffer, lon + buffer, lat + buffer]
        using_point_query = True
    else:
        # Use the provided bounds
        total_bounds = bounds.total_bounds
        using_point_query = False
    
    @fused.cache
    def get_data(total_bounds, file_path, h3_size):
        con = utils.duckdb_connect()
        # DuckDB query to:
        # 1. Convert lat/long to H3 cells
        # 2. Calculate average download speed per cell
        # 3. Filter by geographic bounds
        qr=f'''select  h3_latlng_to_cell(tile_y, tile_x, {h3_size}) as hex, 
                    avg(avg_d_kbps) as metric
        from read_parquet("{file_path}") 
        where 1=1
        and tile_x between {total_bounds[0]} and {total_bounds[2]}
        and tile_y between {total_bounds[1]} and {total_bounds[3]}
        group by 1
        ''' 
        df = con.sql(qr).df()
        return df
    
    # Calculate H3 resolution based on zoom level or use a fixed high resolution for point queries    
    if using_point_query:
        res = 8  # Use high resolution for point queries
    else:
        res_offset = 0
        res = max(min(int(2+bounds.z[0]/1.5),8)-res_offset,2)
    
    df = get_data(total_bounds, file_path, h3_size=res)
    
    # For point queries, find the closest H3 cell and return its speed
    if using_point_query:
        con = utils.duckdb_connect()
        
        # Convert the input lat/lon to an H3 cell
        point_cell_query = f'''
        SELECT h3_latlng_to_cell({lat}, {lon}, {res}) as point_hex
        '''
        point_cell_df = con.sql(point_cell_query).df()
        
        if not point_cell_df.empty:
            point_cell = point_cell_df['point_hex'].iloc[0]
            
            # Find the cell in our results that matches the point's cell
            if 'hex' in df.columns and not df.empty:
                point_speed = df[df['hex'] == point_cell]
                
                if not point_speed.empty:
                    print(f"Speed at location ({lat}, {lon}): {point_speed['metric'].iloc[0]} kbps")
                    point_speed.rename(columns={'metric': 'internet_speed_kbs'}, inplace=True)
                    print(f"{point_speed=}")
                    return point_speed
                else:
                    print(f"No exact match found for location ({lat}, {lon}). Returning all cells in area.")
            else:
                print(f"No data found for location ({lat}, {lon})")
    
    print(df) 
    return df


In [39]:
fused.run(ookla_internet_speed)

,hex,internet_speed_kbs
6,613196570331971583,373333.333333


In [40]:
internet_speed_mcp_metadata = {
    "description": "This example demonstrates how Ookla's mobile performance data can be dynamically processed into an H3 hexagonal grid system. The network metrics are aggregated (averaging download speeds) for H3 hexes at a resolution that adapts based on the zoom level. The performance data comes from Ookla's global speed test infrastructure, capturing real-world mobile network performance across diverse network operators and technologies. The data is stored in Parquet format on S3, structured by year and quarter, allowing for efficient geographic querying and temporal analysis. The resulting hexagonal grid provides a standardized way to visualize and analyze mobile network performance patterns across different geographic scales and regions.",
    "parameters": [
        {
            "name": "lat",
            "type": "float"
        },
        {
            "name": "lon",
            "type": "float"
        }
    ],
}

internet_speed_mcp_metadata

{'description': "This example demonstrates how Ookla's mobile performance data can be dynamically processed into an H3 hexagonal grid system. The network metrics are aggregated (averaging download speeds) for H3 hexes at a resolution that adapts based on the zoom level. The performance data comes from Ookla's global speed test infrastructure, capturing real-world mobile network performance across diverse network operators and technologies. The data is stored in Parquet format on S3, structured by year and quarter, allowing for efficient geographic querying and temporal analysis. The resulting hexagonal grid provides a standardized way to visualize and analyze mobile network performance patterns across different geographic scales and regions.",
 'parameters': [{'name': 'lat', 'type': 'float'},
  {'name': 'lon', 'type': 'float'}]}

In [41]:
# We add this new UDF + mcp_metadata to the same agent
common.save_to_agent(
    agent_json_path=os.path.join(WORKING_DIR, "agents.json"),
    udf=ookla_internet_speed,
    agent_name=AGENT_NAME,
    udf_name="internet_speeds_for_lat_lon",
    mcp_metadata=internet_speed_mcp_metadata,
)

In [42]:
# Let's make sure we created our agent properly, with all our UDFs
agents = json.load(open(os.path.join(WORKING_DIR, "agents.json")))
print(json.dumps(agents, indent=4, sort_keys=True))

{
    "agents": [
        {
            "name": "get_current_time",
            "udfs": [
                "current_utc_time"
            ]
        },
        {
            "name": "fused_docs",
            "udfs": [
                "list_public_udfs",
                "reading_fused_docs"
            ]
        },
        {
            "name": "vancouver_open_data",
            "udfs": [
                "hundred_parks_in_vancouver",
                "electric_vehicle_chargers_in_vancouver",
                "building_permits_in_vancouver",
                "internet_speeds_for_lat_lon"
            ]
        }
    ]
}


Now we can tell Claude we want to use this `vancouver_open_data` Agent, defined in `AGENT_NAME`

In [47]:
AGENT_NAME

'vancouver_open_data'

In [50]:
# Finally, we can select which Agent we want to pass to Claude in our MCP server config
common.generate_local_mcp_config(
    config_path=PATH_TO_CLAUDE_CONFIG,
    agents_list=[AGENT_NAME],
    repo_path=WORKING_DIR,
)

Claude uses a specific config (that you passed under `PATH_TO_CLAUDE_CONFIG`) to know what to run under the hood. This is what we're editing for you each time you change the agent you want to run

In [51]:
# Let's read this Claude Desktop config to see what we're passing
claude_config = json.load(open(PATH_TO_CLAUDE_CONFIG))
print(json.dumps(claude_config, indent=4, sort_keys=True))

{
    "mcpServers": {
        "vancouver_open_data": {
            "args": [
                "run",
                "--directory",
                "/Users/maximelenormand/Library/CloudStorage/Dropbox/Mac/Documents/repos/fused-mcp",
                "main.py",
                "--runtime=local",
                "--udf-names=hundred_parks_in_vancouver,electric_vehicle_chargers_in_vancouver,building_permits_in_vancouver,internet_speeds_for_lat_lon",
                "--name=vancouver_open_data"
            ],
            "command": "uv"
        }
    }
}


## Now let's restart Claude with this new agent!

In [52]:
def restart_claude(claude_path: str = CLAUDE_APP_PATH):
    app_name = claude_path.split("/")[-1]

    try:
        os.system(f"pkill -f '{app_name}'")
        print(f"Killed {app_name}")
        time.sleep(2)  # Wait for shutdown
    except Exception:
        print("Claude wasn't running, so no need to kill it")

    print(f"Restarting {app_name}")
    os.system(f"open -a '{claude_path}'")  # Restart Claude

In [53]:
restart_claude()

Killed Claude.app
Restarting Claude.app
